In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import re
import utils
from bs4 import BeautifulSoup

In [2]:
products = pd.read_csv('data/DreamMarket_master_product.csv')

C:\Users\morga\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# utils.plot_null_val_heatmap(products, 'Null Values of Product Dataset')

## Functions - Add to utils.py

## Review Raw Data

In [4]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130936 entries, 0 to 130935
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   seqNo             39473 non-null   float64
 1   product_name      130936 non-null  object 
 2   category          130936 non-null  object 
 3   description       130936 non-null  object 
 4   shipping_options  130936 non-null  object 
 5   keywords          130936 non-null  object 
 6   seller_name       130936 non-null  object 
 7   price             130936 non-null  object 
 8   payment_method    130936 non-null  object 
 9   sold_since        130936 non-null  object 
 10  ends_in           130936 non-null  object 
 11  quantity_sold     130936 non-null  object 
 12  quantity_left     130936 non-null  object 
 13  refundPolicy      130936 non-null  object 
 14  market_name       130936 non-null  object 
 15  ship_from         130932 non-null  object 
 16  ship_to           13

In [5]:
products.head()

,seqNo,product_name,category,description,shipping_options,keywords,seller_name,price,payment_method,sold_since,ends_in,quantity_sold,quantity_left,refundPolicy,market_name,ship_from,ship_to,idproduct
0,1.0,503 - Forwarding failure (Privoxy@localhost),-,-,-,-,-,-,-,-,-,-,-,-,Dream,-,-,NaN
1,2.0,OG KUSH 1/2 OZ $125 AAAA+,Drugs \n\t\t41793,OG Kush makes up the genetic backbone of West ...,\n\n\n$15.99\t\t\t\t\t\nCANADA WIDE\n\n\n\n\n\...,-,GWLOVE,$125,Escrow Yes,-,-,-,-,-,Dream,CANADA,Worldwide Worldwide,NaN
2,3.0,Dream Market Login - Featured anonymous market...,-,-,-,-,-,-,-,-,-,-,-,-,Dream,-,-,NaN
3,4.0,Every drug available on request,Drugs \n\t\t41804,Every kind of medicine I can arrange . Send me...,-,-,Hectorstuff,$1,Escrow Yes,-,-,-,-,-,Dream,dutch,Worldwide Worldwide,NaN
4,5.0,"Debug, Learn how crack windows",Digital Goods \n\t\t29334,"Debug, Learn how crack windows",-,-,darkmarket,$1,Escrow Yes,-,-,-,-,-,Dream,Worldwide,Worldwide Worldwide,NaN


In [6]:
cols_to_review = ['sold_since', 'ends_in', 'quantity_sold', 'quantity_left', 'refundPolicy', 'keywords']
for x in cols_to_review:
    print('{}: {}'.format(x, products[x].unique()))

sold_since: ['-']
ends_in: ['-']
quantity_sold: ['-']
quantity_left: ['-']
refundPolicy: ['-']
keywords: ['-']


## Text Cleaning

In [7]:
products_v2 = products.drop(cols_to_review, axis=1)

In [8]:
products_v2['is_2016'] = products_v2['idproduct'].isnull()
products['year_sold'] = products_v2['is_2016'].apply(lambda x: 2016 if True else 2017)
products_v2.drop('is_2016', axis=1, inplace=True)
products_v2['adj_price'] = products_v2['price'].apply(lambda x: utils.convert_price_to_float(x))

In [9]:
products_v2['category_revised'] = products_v2['category'].str.replace(r'\n', '')
products_v2['category_revised'] = products_v2['category_revised'].str.replace('\t', '')
# products_v2['category_revised'] = products_v2['category_revised'].apply(lambda x: utils.replace_dash_with_nan(x))
products_v2['category_str'] = products_v2['category_revised'].str.findall(r'[a-zA-Z ]')

products_v2['category_str'] = products_v2['category_str'].apply(lambda x: ''.join(x))
products_v2['category_num'] = products_v2['category_revised'].str.findall(r'[0-9 ]')
products_v2['category_num'] = products_v2['category_num'].apply(lambda x: ''.join(x).replace(' ', ''))

In [12]:
products_v2['category_revised'] = products_v2['category_revised'].apply(lambda x: utils.replace_dash_with_nan(x))

In [13]:
products_v2.head()

,seqNo,product_name,category,description,shipping_options,seller_name,price,payment_method,market_name,ship_from,ship_to,idproduct,adj_price,category_revised,category_str,category_num
0,1.0,503 - Forwarding failure (Privoxy@localhost),-,-,-,-,-,-,Dream,-,-,NaN,NaN,NaN,,
1,2.0,OG KUSH 1/2 OZ $125 AAAA+,Drugs \n\t\t41793,OG Kush makes up the genetic backbone of West ...,\n\n\n$15.99\t\t\t\t\t\nCANADA WIDE\n\n\n\n\n\...,GWLOVE,$125,Escrow Yes,Dream,CANADA,Worldwide Worldwide,NaN,125.0,Drugs 41793,Drugs,41793
2,3.0,Dream Market Login - Featured anonymous market...,-,-,-,-,-,-,Dream,-,-,NaN,NaN,NaN,,
3,4.0,Every drug available on request,Drugs \n\t\t41804,Every kind of medicine I can arrange . Send me...,-,Hectorstuff,$1,Escrow Yes,Dream,dutch,Worldwide Worldwide,NaN,1.0,Drugs 41804,Drugs,41804
4,5.0,"Debug, Learn how crack windows",Digital Goods \n\t\t29334,"Debug, Learn how crack windows",-,darkmarket,$1,Escrow Yes,Dream,Worldwide,Worldwide Worldwide,NaN,1.0,Digital Goods 29334,Digital Goods,29334


In [10]:
#products['ship_from_proper'] = products['ship_from'].apply(lambda x: x.title())
products['ship_to_proper'] = products['ship_to'].apply(lambda x: x.title())
products['category_str'].fillna(' ', inplace=True)
products['category_num'].fillna(' ', inplace=True)

KeyError: 'category_str'

In [ ]:
products['shipping_options_revised'] = products['shipping_options'].str.replace(r'\n', ' ')
products['shipping_options_revised'] = products['shipping_options_revised'].str.replace('\t', ' ')

In [ ]:
products.head()

## EDA

In [ ]:
utils.stats_summary(products['adj_price'])

In [ ]:
plt.figure(figsize=(25,8))
less_than_100 = products[products['adj_price'] < 100]['adj_price']
sns.distplot(less_than_100)
plt.title('Price Distribution of Products Less Than 100', fontsize= 20);
print('# - Goods Less Than $100: ', len(less_than_100))
print('% - Total Good Less Than $100: {:.1%}'.format(len(less_than_100) / products.shape[0]))